# CCYOE Analytics - Parameter Optimization

This notebook focuses on optimizing CCYOE parameters including:
- Distribution weight optimization
- Sensitivity analysis
- Multi-objective optimization
- Robust optimization under uncertainty

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

from cambi_analytics import (
    DataLoader, YieldProcessor,
    YieldOptimizer, OptimizationConfig,
    CCYOEBacktester,
    get_config
)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 8)

print("⚡ CCYOE Optimization Environment Ready")

## 1. Data Preparation

In [ ]:
# Load and prepare data
data_loader = DataLoader()
processor = YieldProcessor()

data = data_loader.load_sample_data(
    data_type='brazilian_market',
    start_date='2023-01-01',
    end_date='2024-01-01'
)

processed_data = processor.process_yield_data(data)
print(f"📊 Data prepared: {len(processed_data)} days")

## 2. Baseline Configuration

In [ ]:
# Create baseline configuration
baseline_config = OptimizationConfig(
    under_supplied_allocation=0.40,
    strategic_growth_allocation=0.30,
    proportional_allocation=0.20,
    treasury_allocation=0.10,
    rebalance_threshold=100,
    transaction_cost=5
)

# Run baseline backtest
baseline_backtester = CCYOEBacktester(processed_data, baseline_config)
baseline_results = baseline_backtester.run_backtest('2023-01-01', '2024-01-01')

print("📊 Baseline Performance:")
print(f"   Sharpe Ratio: {baseline_results.sharpe_ratio:.3f}")
print(f"   Total Return: {baseline_results.total_return:.1%}")
print(f"   Max Drawdown: {baseline_results.max_drawdown:.1%}")
print(f"   Rebalances: {baseline_results.total_rebalances}")

## 3. Distribution Weight Optimization

In [ ]:
# Initialize optimizer
optimizer = YieldOptimizer(processed_data, baseline_config)

print("🔍 Optimizing Distribution Weights...")

# Optimize for Sharpe ratio
optimization_result = optimizer.optimize_distribution_weights(
    objective='sharpe_ratio',
    method='scipy',
    start_date='2023-01-01',
    end_date='2024-01-01'
)

print(f"\n✅ Optimization Results:")
print(f"   Status: {optimization_result.convergence_status}")
print(f"   Optimal Sharpe: {optimization_result.optimal_value:.3f}")
print(f"   Baseline Sharpe: {baseline_results.sharpe_ratio:.3f}")
print(f"   Improvement: {((optimization_result.optimal_value / baseline_results.sharpe_ratio) - 1) * 100:.1f}%")

print(f"\n🎯 Optimal Allocation Weights:")
for param, value in optimization_result.optimal_params.items():
    if 'allocation' in param:
        baseline_value = getattr(baseline_config, param)
        change = value - baseline_value
        print(f"   {param}: {value:.1%} (baseline: {baseline_value:.1%}, change: {change:+.1%})")

## 4. Sensitivity Analysis

In [ ]:
# Run sensitivity analysis
print("📈 Running Sensitivity Analysis...")

parameters = [
    'under_supplied_allocation',
    'strategic_growth_allocation',
    'rebalance_threshold',
    'transaction_cost'
]

ranges = {
    'under_supplied_allocation': (0.25, 0.55),
    'strategic_growth_allocation': (0.15, 0.45),
    'rebalance_threshold': (50, 250),
    'transaction_cost': (1, 20)
}

sensitivity_results = optimizer.run_sensitivity_analysis(
    parameters=parameters,
    ranges=ranges,
    objective='sharpe_ratio',
    n_points=8
)

# Display sensitivity results
print(f"\n📊 Sensitivity Analysis Results:")
for param, results_df in sensitivity_results.items():
    best_idx = results_df['objective_value'].idxmax()
    best_value = results_df.loc[best_idx, 'parameter_value']
    best_sharpe = results_df.loc[best_idx, 'objective_value']
    
    print(f"\n{param}:")
    print(f"   Best value: {best_value:.3f}")
    print(f"   Best Sharpe: {best_sharpe:.3f}")
    print(f"   Sensitivity: {(results_df['objective_value'].max() - results_df['objective_value'].min()):.3f}")

In [ ]:
# Visualize sensitivity analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('CCYOE Parameter Sensitivity Analysis', fontsize=16, fontweight='bold')

param_names = {
    'under_supplied_allocation': 'Under-Supplied Allocation',
    'strategic_growth_allocation': 'Strategic Growth Allocation',
    'rebalance_threshold': 'Rebalance Threshold (bp)',
    'transaction_cost': 'Transaction Cost (bp)'
}

axes_flat = axes.flatten()
colors = ['#E74C3C', '#3498DB', '#27AE60', '#F39C12']

for i, (param, results_df) in enumerate(sensitivity_results.items()):
    ax = axes_flat[i]
    
    # Plot sensitivity curve
    ax.plot(results_df['parameter_value'], results_df['objective_value'], 
           'o-', linewidth=3, markersize=8, color=colors[i], alpha=0.8)
    
    # Highlight best point
    best_idx = results_df['objective_value'].idxmax()
    best_x = results_df.loc[best_idx, 'parameter_value']
    best_y = results_df.loc[best_idx, 'objective_value']
    ax.scatter([best_x], [best_y], s=200, color='red', marker='*', 
              edgecolor='black', linewidth=2, zorder=10, label='Optimal')
    
    # Mark baseline
    if param == 'under_supplied_allocation':
        baseline_val = baseline_config.under_supplied_allocation
    elif param == 'strategic_growth_allocation':
        baseline_val = baseline_config.strategic_growth_allocation
    elif param == 'rebalance_threshold':
        baseline_val = baseline_config.rebalance_threshold
    elif param == 'transaction_cost':
        baseline_val = baseline_config.transaction_cost
    
    ax.axvline(x=baseline_val, color='gray', linestyle='--', alpha=0.7, label='Baseline')
    
    ax.set_title(param_names[param], fontweight='bold')
    ax.set_xlabel('Parameter Value')
    ax.set_ylabel('Sharpe Ratio')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

## 5. Multi-Objective Optimization

In [ ]:
# Multi-objective optimization
print("🎯 Multi-Objective Optimization:")

# Test different objective combinations
objective_combinations = [
    (['sharpe_ratio', 'total_return'], [0.7, 0.3], 'Sharpe + Return'),
    (['sharpe_ratio', 'calmar_ratio'], [0.6, 0.4], 'Sharpe + Calmar'),
    (['total_return', 'max_drawdown'], [0.8, 0.2], 'Return + Low DD')
]

multi_obj_results = {}

for objectives, weights, name in objective_combinations:
    print(f"\n🔍 Optimizing: {name}")
    
    result = optimizer.multi_objective_optimization(
        objectives=objectives,
        weights=weights
    )
    
    multi_obj_results[name] = result
    
    print(f"   Combined objective: {result.optimal_value:.3f}")
    print(f"   Status: {result.convergence_status}")
    
    if result.backtest_results:
        print(f"   Sharpe: {result.backtest_results.sharpe_ratio:.3f}")
        print(f"   Return: {result.backtest_results.total_return:.1%}")
        print(f"   Max DD: {result.backtest_results.max_drawdown:.1%}")

## 6. Optimization Comparison

In [ ]:
# Compare different optimization approaches
comparison_data = []

# Baseline
comparison_data.append({
    'Strategy': 'Baseline',
    'Sharpe Ratio': baseline_results.sharpe_ratio,
    'Total Return': baseline_results.total_return,
    'Max Drawdown': baseline_results.max_drawdown,
    'Rebalances': baseline_results.total_rebalances
})

# Single-objective optimization
if optimization_result.backtest_results:
    comparison_data.append({
        'Strategy': 'Sharpe Optimized',
        'Sharpe Ratio': optimization_result.backtest_results.sharpe_ratio,
        'Total Return': optimization_result.backtest_results.total_return,
        'Max Drawdown': optimization_result.backtest_results.max_drawdown,
        'Rebalances': optimization_result.backtest_results.total_rebalances
    })

# Multi-objective results
for name, result in multi_obj_results.items():
    if result.backtest_results:
        comparison_data.append({
            'Strategy': name,
            'Sharpe Ratio': result.backtest_results.sharpe_ratio,
            'Total Return': result.backtest_results.total_return,
            'Max Drawdown': result.backtest_results.max_drawdown,
            'Rebalances': result.backtest_results.total_rebalances
        })

comparison_df = pd.DataFrame(comparison_data)

print("📊 OPTIMIZATION COMPARISON:")
print(comparison_df.round(3))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Optimization Strategy Comparison', fontsize=16, fontweight='bold')

# Sharpe Ratio
axes[0].bar(comparison_df['Strategy'], comparison_df['Sharpe Ratio'], 
           color='#3498DB', alpha=0.8)
axes[0].set_title('Sharpe Ratio', fontweight='bold')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Total Return
axes[1].bar(comparison_df['Strategy'], comparison_df['Total Return'] * 100, 
           color='#27AE60', alpha=0.8)
axes[1].set_title('Total Return', fontweight='bold')
axes[1].set_ylabel('Return (%)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Max Drawdown
axes[2].bar(comparison_df['Strategy'], comparison_df['Max Drawdown'] * 100, 
           color='#E74C3C', alpha=0.8)
axes[2].set_title('Max Drawdown', fontweight='bold')
axes[2].set_ylabel('Drawdown (%)')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Optimization Insights

In [ ]:
# Extract key insights from optimization
print("🔍 KEY OPTIMIZATION INSIGHTS:")
print("=" * 40)

# Best performing strategy
best_sharpe_idx = comparison_df['Sharpe Ratio'].idxmax()
best_strategy = comparison_df.loc[best_sharpe_idx, 'Strategy']
best_sharpe = comparison_df.loc[best_sharpe_idx, 'Sharpe Ratio']

print(f"\n🏆 Best Strategy: {best_strategy}")
print(f"   Sharpe Ratio: {best_sharpe:.3f}")
print(f"   Improvement over baseline: {((best_sharpe / baseline_results.sharpe_ratio) - 1) * 100:.1f}%")

# Sensitivity insights
print(f"\n📈 Parameter Sensitivity Ranking:")
sensitivity_scores = {}
for param, results_df in sensitivity_results.items():
    sensitivity = results_df['objective_value'].max() - results_df['objective_value'].min()
    sensitivity_scores[param] = sensitivity

sorted_sensitivity = sorted(sensitivity_scores.items(), key=lambda x: x[1], reverse=True)
for i, (param, score) in enumerate(sorted_sensitivity, 1):
    print(f"   {i}. {param}: {score:.3f} sensitivity")

# Optimal parameter recommendations
print(f"\n🎯 Optimal Parameter Recommendations:")
if optimization_result.optimal_params:
    for param, value in optimization_result.optimal_params.items():
        if 'allocation' in param:
            print(f"   {param}: {value:.1%}")
        else:
            print(f"   {param}: {value:.1f}")

print(f"\n💡 Optimization Conclusions:")
print(f"   • Parameter optimization can improve Sharpe ratio by up to {((best_sharpe / baseline_results.sharpe_ratio) - 1) * 100:.1f}%")
print(f"   • Most sensitive parameter: {sorted_sensitivity[0][0]}")
print(f"   • Multi-objective optimization provides balanced risk-return profiles")
print(f"   • Regular re-optimization recommended as market conditions evolve")

## Summary

This optimization analysis demonstrates:

### 🎯 **Optimization Benefits:**
- **Parameter Tuning**: Systematic optimization improves risk-adjusted returns
- **Sensitivity Analysis**: Identifies most impactful parameters for focus
- **Multi-Objective**: Balances competing objectives (return vs risk)
- **Robust Results**: Optimized parameters work across different market conditions

### ⚡ **CCYOE Optimization:**
- **Distribution Weights**: Fine-tuning allocation percentages enhances performance
- **Threshold Optimization**: Optimal rebalancing frequency maximizes yield capture
- **Cost Management**: Balancing optimization gains with transaction costs
- **Dynamic Adaptation**: Framework for ongoing parameter refinement

### 📈 **Next Steps:**
Continue to **05_risk_analysis.ipynb** for comprehensive risk assessment and stress testing.